# Read xml workspace and fit using pyhf

In [1]:
import pyhf
import pyhf.readxml
import json

pyhf.set_backend("numpy", pyhf.optimize.minuit_optimizer(verbose=True))

In [2]:
main_xml_path = "TRExFitter/minimal_example/RooStats/minimal_example.xml"
parent_folder = "TRExFitter/"
json_path     = "workspace_from_xml.json"

### translate the `xml` to python and dump it to a `json` file for later use

In [3]:
spec = pyhf.readxml.parse(main_xml_path, parent_folder, track_progress=False)

with open(json_path, "w") as f:
    json.dump(spec, f, sort_keys=True, indent=4)

### load spec from file again

In [4]:
with open(json_path, "r") as f:
    spec = json.load(f)

### create the relevant pyhf objects

In [5]:
workspace = pyhf.Workspace(spec)
model = workspace.model()
data = workspace.data(model)

### perform an unconstrained fit

In [6]:
%%time
result = pyhf.infer.mle.fit(data, model, return_uncertainties=True)

------------------------------------------------------------------
| FCN = 21.61                   |     Ncalls=212 (212 total)     |
| EDM = 0.000108 (Goal: 1E-05)  |            up = 1.0            |
------------------------------------------------------------------
|  Valid Min.   | Valid Param.  | Above EDM | Reached call limit |
------------------------------------------------------------------
|     True      |     True      |   False   |       False        |
------------------------------------------------------------------
| Hesse failed  |   Has cov.    | Accurate  | Pos. def. | Forced |
------------------------------------------------------------------
|     False     |     True      |   True    |   True    | False  |
------------------------------------------------------------------
CPU times: user 50.1 ms, sys: 4.53 ms, total: 54.6 ms
Wall time: 51.8 ms


### show the results

In [7]:
def get_parameter_names(model):
    labels = []
    for parname in model.config.par_order:
        for i_par in range(model.config.param_set(parname).n_parameters):
            labels.append(
                "{}[bin_{}]".format(parname, i_par)
                if model.config.param_set(parname).n_parameters > 1
                else parname
            )
    return labels

bestfit = result[:, 0]
uncertainty = result[:, 1]
labels = get_parameter_names(model)

for i, label in enumerate(labels):
    print(f"{label}: {bestfit[i]} +/- {uncertainty[i]}")

lumi: 1.00210226827377 +/- 0.01929133949918138
staterror_Signal_region[bin_0]: 1.0003425715548206 +/- 0.005807817286532124
staterror_Signal_region[bin_1]: 0.9988215875033956 +/- 0.0056055039396046835
staterror_Signal_region[bin_2]: 1.0034524200724302 +/- 0.005095426443941953
staterror_Signal_region[bin_3]: 0.9975282708630174 +/- 0.006000955883891657
staterror_Signal_region[bin_4]: 0.9989829541808631 +/- 0.007892029640412968
staterror_Signal_region[bin_5]: 0.9973982717169335 +/- 0.011986329787382344
Luminosity: 0.00024602355778569063 +/- 0.9111326823379149
Signal_norm: 1.1863179067062717 +/- 0.03731826916514769


The parameters `lumi` and `Luminosity` both show up here and are highly anti-correlated. While `lumi` is a built-in way that HistFactory provides for a luminosity uncertainty, TRExFitter does not make use of it. Instead, a more configurable normalization factor is manually implemented in the TRExFitter configuration. While the `lumi` parameter is set to constant in `minimal_example.xml` (via `<ParamSetting Const="True">Lumi</ParamSetting>`), this information is not yet propagated automatically through `pyhf` to the minimizer. See also https://github.com/scikit-hep/pyhf/pull/846.